## This notebook utilizes the tool to make a refine-sample-coarsen model topography 

In [1]:
import GMesh_torch
import netCDF4
import numpy as np
import matplotlib.pyplot as plt
import time
%matplotlib inline
import torch
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cuda')

## GridMesh Class

## Read GEBCO dataset for topography

In [2]:
# Read topo data
#! cp -n /archive/gold/datasets/topography/GEBCO_2023/GEBCO_2023.nc .
#! ln -s /home/Niki.Zadeh/datasets .datasets
#source_datafile='.datasets/GEBCO_2020.nc'
source_datafile='.datasets/GEBCO_2023.nc'
with netCDF4.Dataset(source_datafile) as nc:
    topo_lons = nc.variables['lon'][:].filled(0.)
    topo_lats = nc.variables['lat'][:].filled(0.)
    topo_elvs = nc.variables['elevation'][:,:].filled(0.)

In [3]:
t_topo_lons=torch.from_numpy(topo_lons).to(device)
t_topo_lats=torch.from_numpy(topo_lats).to(device)
t_topo_elvs=torch.from_numpy(topo_elvs).to(device)

In [4]:
src_topo_global = GMesh_torch.UniformEDS( t_topo_lons, t_topo_lats, t_topo_elvs, device)
src_topo_global

<UniformEDS 86400 x 43200 (27.810Gb)
lon = <RegularCoord n=86400, dx=0.004166666666666667, rdx=240.0, x0=-180.0, io=-43200, rem=0.0, is-ie=0-86400, periodic=True>
h:tensor([-179.9979, -179.9938, -179.9896,  ...,  179.9896,  179.9937,
         179.9979], device='cuda:0', dtype=torch.float64)
q:tensor([-180.0000, -179.9958, -179.9917,  ...,  179.9917,  179.9958,
         180.0000], device='cuda:0')
lat = <RegularCoord n=43200, dx=0.004166666666666667, rdx=240.0, x0=-90, io=-21600, rem=0.0, is-ie=0-43200, periodic=False>
h:tensor([-89.9979, -89.9938, -89.9896,  ...,  89.9896,  89.9938,  89.9979],
       device='cuda:0', dtype=torch.float64)
q:tensor([-90.0000, -89.9958, -89.9917,  ...,  89.9917,  89.9958,  90.0000],
       device='cuda:0')
data.shape = torch.Size([43200, 86400])
data = tensor([[ 2829,  2829,  2829,  ...,  2830,  2830,  2830],
        [ 2830,  2830,  2830,  ...,  2830,  2830,  2830],
        [ 2831,  2831,  2831,  ...,  2832,  2832,  2832],
        ...,
        [-4221, -42

In [5]:
print(' topography grid array shapes: ' , topo_lons.shape,topo_lats.shape,topo_elvs.shape)
print(' topography longitude range:',topo_lons.min(),topo_lons.max())
print(' topography latitude range:',topo_lats.min(),topo_lats.max())
#print(' Is mesh uniform?', GMesh.is_mesh_uniform( topo_lons, topo_lats ) )
#GEBCO_2014_2D.nc
# topography grid array shapes:  (43200,) (21600,) (21600, 43200)
# topography longitude range: -299.995833333 59.9958333333
# topography latitude range: -89.9958333333 89.9958333333
# Is mesh uniform? True
#GEBCO_2020.nc
# topography grid array shapes:  (86400,) (43200,) (43200, 86400)
# topography longitude range: -299.9979166666667 59.99791666666667
# topography latitude range: -89.99791666666667 89.99791666666667
# Is mesh uniform? True

 topography grid array shapes:  (86400,) (43200,) (43200, 86400)
 topography longitude range: -179.99791666666667 179.99791666666667
 topography latitude range: -89.99791666666667 89.99791666666667


## Read the target grid

Terget grid is the underlying finite element 2D supergrid to be used in the Ocean model. Here we choose a 1/4 degree Mercator grid  generated by using the [grid_generation tool](https://github.com/nikizadehgfdl/grid_generation). 

In [6]:
# Read target mesh
#OM5 grid
#scp /archive/jpk/datasets/OM5/OM5_025/v20240311/ocean_hgrid.nc Niki.Zadeh@lscamd50-d:/home/Niki.Zadeh/datasets/ocean_hgrid_OM5p25v20240311.nc
#! ln -s /home/Niki.Zadeh/datasets .datasets
ocean_hgrid_file='.datasets/ocean_hgrid.Merc.4deg.nc'
#NtileI, NtileJ, max_refinement = 30, 1, 9
ocean_hgrid_file='.datasets/OM5p25v20240311_jpk/ocean_hgrid_OM5p25v20240311.nc'
with netCDF4.Dataset(ocean_hgrid_file) as nc:
    lon=nc.variables['x'][::2,::2]
    lat=nc.variables['y'][::2,::2]
t_lon=torch.from_numpy(lon).to(device)
t_lat=torch.from_numpy(lat).to(device)
targG = GMesh_torch.GMesh_torch( lon=t_lon, lat=t_lat, device=device )
targG

<GMesh_torch nj:1161 ni:1440 shape:(1161,1440)>

In [7]:
#verify cuda-in cuda-out functions
def lonlat_to_XYZ(lon, lat):
    """Private method. Returns 3d coordinates (X,Y,Z) of spherical coordiantes (lon,lat)."""
    deg2rad = torch.pi/180.
    lonr,latr = deg2rad*lon, deg2rad*lat
    return torch.cos( latr )*torch.cos( lonr ), torch.cos( latr )*torch.sin( lonr ), torch.sin( latr )

X,Y,Z=lonlat_to_XYZ(targG.lon,targG.lat)
targG.lon.type(),X.type()


('torch.cuda.DoubleTensor', 'torch.cuda.DoubleTensor')

In [8]:
GMesh_torch.pfactor( targG.ni ), GMesh_torch.pfactor( targG.nj )

([5, 3, 3, 2, 2, 2, 2, 2], [43, 3, 3, 3])

In [9]:
targG.lon.shape[0],targG.lon[0,0],t_lon[0,0]

(1162,
 tensor(-300., device='cuda:0', dtype=torch.float64),
 tensor(-300., device='cuda:0', dtype=torch.float64))

In [10]:
#import create_topog_refinedSampling as topotool
#jllc,illc,status1=topotool.get_indices1D(topo_lons, topo_lats ,targG.lon[0,0] ,targG.lat[0,0])
#jurc,iurc,status2=topotool.get_indices1D(topo_lons, topo_lats ,targG.lon[0,-1],targG.lat[-1,0])

In [11]:
def convol( levels, h, f, verbose=False ):
    """Coarsens the product of h*f across all levels"""
    levels[-1].height = ( h * f ).reshape(levels[-1].nj,levels[-1].ni)
    for k in range( len(levels) - 1, 0, -1 ):
        if verbose: print('Coarsening {} -> {}'.format(k,k-1))
        levels[k].coarsenby2( levels[k-1] )
    return levels[0].height  

def rough( levels, h , device, h2min=1.e-7):
    """Calculates both mean of H, and variance of H relative to a plane"""
    # Construct weights for moment calculations
    nx = 2**( len(levels) - 1 )
    x = ( np.arange(nx) - ( nx - 1 ) /2 ) * np.sqrt( 12 / ( nx**2 - 1 ) ) # This formula satisfies <x>=0 and <x^2>=1
    x = torch.from_numpy(x).to(device)
    X, Y = torch.meshgrid( x, x )
    X, Y = X.reshape(1,nx,1,nx), Y.reshape(1,nx,1,nx)
    h = torch.reshape(h,(levels[0].nj,nx,levels[0].ni,nx)).to(device) #Why is this not already on device? Input h is on device!
    # Now calculate moments
    H2 = convol( levels, h, h ) # mean of h^2
    HX = convol( levels, h, X ) # mean of h * x
    HY = convol( levels, h, Y ) # mean of h * y
    H = convol( levels, h, torch.ones((1,nx,1,nx)).to(device)) # mean of h = mean of h * 1
    # The variance of deviations from the plane = <h^2> - <h>^2 - <h*x>^2 - <h*y>^2 given <x>=<y>=0 and <x^2>=<y^2>=1
    return H, H2 - H**2 - HX**2 - HY**2 + torch.tensor((h2min))

def rough0( levels, h, h2min=1.e-7):
    """Calculates both mean of H, and variance of H relative to a plane"""
    # Construct weights for moment calculations
    nx = 2**( len(levels) - 1 )
    x = ( np.arange(nx) - ( nx - 1 ) /2 ) * np.sqrt( 12 / ( nx**2 - 1 ) ) # This formula satisfies <x>=0 and <x^2>=1
    X, Y = np.meshgrid( x, x )
    X, Y = X.reshape(1,nx,1,nx), Y.reshape(1,nx,1,nx)
    h = h.reshape(levels[0].nj,nx,levels[0].ni,nx)
    # Now calculate moments
    H2 = convol( levels, h, h ) # mean of h^2
    HX = convol( levels, h, X ) # mean of h * x
    HY = convol( levels, h, Y ) # mean of h * y
    H = convol( levels, h, np.ones((1,nx,1,nx)) ) # mean of h = mean of h * 1
    # The variance of deviations from the plane = <h^2> - <h>^2 - <h*x>^2 - <h*y>^2 given <x>=<y>=0 and <x^2>=<y^2>=1
    return H, H2 - H**2 - HX**2 - HY**2 + h2min


In [12]:


def do_RSC_new(targG,src_topo_global,NtileI=1, NtileJ=1, max_refinement=10, device='cpu'):
    """Apply the RSC algoritm using a fixed number of refinements max_refinement"""
    di, dj = targG.ni // NtileI, targG.nj // NtileJ
    assert di*NtileI == targG.ni
    assert dj*NtileJ == targG.nj
    print('window size dj,di =',dj,di,'full model nj,ni=',targG.nj, targG.ni)
    Hcnt = np.zeros((targG.nj, targG.ni)) # Diagnostic: counting which cells we are working on
    Htarg, H2targ = np.zeros((targG.nj, targG.ni)), np.zeros((targG.nj, targG.ni))
    for j in range(NtileJ ):
        csj, sj = slice( j*dj, (j+1)*dj ), slice( j*dj, (j+1)*dj+1 )
        for i in range(NtileI ):
            csi, si = slice( i*di, (i+1)*di ), slice( i*di, (i+1)*di+1 ) # Slices of target grid
            Hcnt[csj,csi] = Hcnt[csj,csi] + 1 # Diagnostic: counting which cells we are working on
            G = GMesh_torch.GMesh_torch( lon=targG.lon[sj,si], lat=targG.lat[sj,si],device=device )
            print('J,I={},{} {:.1f}%, {}\n   window lon={}:{}, lat={}:{}\n   jslice={}, islice={}'.format( \
                j, i, 100*(j*NtileI+i)/(NtileI*NtileJ), G, G.lon.min(), G.lon.max(), G.lat.min(), G.lat.max(), sj, si ))
            levels = G.refine_loop( src_topo_global, resolution_limit=False, fixed_refine_level=max_refinement, timers=False, verbose=False, device=device)
            ## Use nearest neighbor topography to populate the finest grid
            levels[-1].project_source_data_onto_target_mesh( src_topo_global )
            ## Now recursively coarsen
            h, h2 = rough( levels, levels[-1].height,device=device )
            # Store window in final array
            Htarg[csj,csi]  = h.cpu()
            H2targ[csj,csi] = h2.cpu()
    print( Hcnt.min(), Hcnt.max(), '<-- should both be 1 for full model' )
    return  Htarg, H2targ
    
st = time.time()
#Test grid
#NtileI, NtileJ, max_refinement = 10, 1, 9
#OM5p25v20240311
#NtileI, NtileJ, max_refinement = 3*2, 43*3*3, 7
NtileI, NtileJ, max_refinement = 3*2*2*2, 43*3*3, 8
NtileI, NtileJ, max_refinement = 12, 43, 8
Htarg, H2targ= do_RSC_new(targG,src_topo_global,NtileI, NtileJ, max_refinement,device=device)    
et = time.time() - st
print('RSC loop time:', et, 'seconds')


window size dj,di = 27 120 full model nj,ni= 1161 1440
J,I=0,0 0.0%, <GMesh_torch nj:27 ni:120 shape:(27,120)>
   window lon=-300.0:-270.0, lat=-88.57:-85.91759174511695
   jslice=slice(0, 28, None), islice=slice(0, 121, None)


/home/Niki.Zadeh/.conda/envs/plattorch_ext/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /workspace/croot/pytorch_1692381528408/work/aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


J,I=0,1 0.2%, <GMesh_torch nj:27 ni:120 shape:(27,120)>
   window lon=-270.0:-240.0, lat=-88.57:-85.91759174511695
   jslice=slice(0, 28, None), islice=slice(120, 241, None)
J,I=0,2 0.4%, <GMesh_torch nj:27 ni:120 shape:(27,120)>
   window lon=-240.0:-210.0, lat=-88.57:-85.91759174511695
   jslice=slice(0, 28, None), islice=slice(240, 361, None)
J,I=0,3 0.6%, <GMesh_torch nj:27 ni:120 shape:(27,120)>
   window lon=-210.0:-180.0, lat=-88.57:-85.91759174511695
   jslice=slice(0, 28, None), islice=slice(360, 481, None)
J,I=0,4 0.8%, <GMesh_torch nj:27 ni:120 shape:(27,120)>
   window lon=-180.0:-150.0, lat=-88.57:-85.91759174511695
   jslice=slice(0, 28, None), islice=slice(480, 601, None)
J,I=0,5 1.0%, <GMesh_torch nj:27 ni:120 shape:(27,120)>
   window lon=-150.0:-120.0, lat=-88.57:-85.91759174511695
   jslice=slice(0, 28, None), islice=slice(600, 721, None)
J,I=0,6 1.2%, <GMesh_torch nj:27 ni:120 shape:(27,120)>
   window lon=-120.0:-90.0, lat=-88.57:-85.91759174511695
   jslice=slice(

In [13]:
#OM5p25v20240311 topography
#torch CPU, rf=7 RSC loop time:  1557.25546169281 seconds
#
#numpy CPU, rf=8 RSC loop time: 60652.4 seconds ~ 17.8 hrs
#torch CPU, rf=8 RSC loop time:  5834.2 seconds ~  1.6 hrs
#torch GPU, rf=8 RSC loop time:  3093.6 seconds ~  0.9 hrs
#torch GPU, rf=8 RSC loop time:  3246.3
#After optimization to ensure data remains on GPU
#torch GPU, rf=8 RSC loop time:   317.8 seconds ~  6 minutes 
#NtileI, NtileJ, max_refinement = 12, 43, 8
#torch GPU, rf=8 RSC loop time:   272.0 seconds
#Test grid ocean_hgrid.Merc.4deg.nc and GEBCO2020
#After every refinement the number of hits increases by almost a factor of 4. 
#So when more than 25% is hit, upon a refinement the resolution of refined grid 
# becomes more than the resolution of the source and each source point
# gets hit multiple times and the algorithm should be considered as converged.
#
#NtileI=NtileJ=1
#numpy cpu
#rf=8  Hit  253624320  out of  2688360168  cells,    9.4341644776221  percent, time: 141.05 seconds
#rf=9  Hit 1014497280  out of  2688508800  cells,   37.7345716703624  percent, time: 570.34 seconds 
#torch cpu
#rf=6  Hit   15851520  out of  2687952352  cells,    0.5897247392873  percent, time:   1.45 seconds
#rf=8  Hit  253624320  out of  2688297936  cells,    9.4343828711699  percent, time:   8.28 seconds
#rf=9  Hit 1014497280  out of  2688384332  cells,   37.7363187221550  percent, time:  20.34 seconds
#rf=10 Hit 4057989120  out of  3175139396  cells,  127.8050697588963  percent, time: 153.27 seconds
#torch gpu
#rf=8  Hit  253624320  out of  -1606669360  cells,  -15.785719595723167  percent time: 5.298048973083496 seconds
#rf=9 OutOfMemoryError: CUDA out of memory. Tried to allocate 3.78 GiB (GPU 0; 31.73 GiB total capacity; 28.37 GiB already allocated; 2.97 GiB free; 28.40 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF
#NtileI=10, NtileJ=1
#rf=9  Hit  101449728  out of  268850880  cells,  37.7345716703624  percent,  time: 20.95916986465454 seconds  
#rf=9  NonVerbose, RSC loop time: 19.9secs
#rf=10 OutOfMemoryError: CUDA out of memory. Tried to allocate 1.51 GiB (GPU 0; 31.73 GiB total capacity; 27.49 GiB already allocated; 1.46 GiB free; 29.91 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF
#NtileI=30, NtileJ=1
#gpu: rf=10 Hit  135266304  out of  89619840  cells,  150.93343616770574  percent, time:  87.05817294120789 seconds
#cpu: rf=10 Hit  135266304  out of  89619840  cells,  150.93343616770574  percent, time: 120.57575964927673 seconds
#04/12/24
#NtileI, NtileJ, max_refinement = 10, 1, 9
#                            notebook |  script
#torchCPU RSC loop time(secs): 23.9   |  22.7
#torchGPU RSC loop time(secs): 19.9   |  24.2 
#After optimizing to ensure data remains on device
#torchGPU RSC loop time(secs): 10.1   |  10.55
#torchCPU RSC loop time(secs):        |  24.9
#After optimizing rough() to ensure data remains on device
#torchGPU RSC loop time(secs):  2.6   |   2.6
#torchCPU RSC loop time(secs):        |  19.8
###nvtop logs
#                GPU     GPU   MEM    CPU  HOST MEM
#Start  RSC loop 0%   7486MiB  23%     0%   7845MiB
#End of RSC loop 0%  14360MiB  44%     0%   8497MiB 
#rerun the RSC loop cell
#End of RSC loop 0%  14360MiB  44%     0%   8575MiB
#rerun the RSC loop cell and return Glist
#End of RSC loop 0%  14360MiB  44%     0%   9570MiB
#End of RSC loop 0%  14360MiB  44%     0%  10584MiB
#After optimizing 
#End of RSC loop 0%  15280MiB  47%     0%   8004MiB

In [14]:
with netCDF4.Dataset('new_topo_OM5_grid_r{}_{}x{}.nc'.format(max_refinement, NtileI, NtileJ),'w','clobber') as nc:
    nx = nc.createDimension('nx', Htarg.shape[1])
    ny = nc.createDimension('ny', Htarg.shape[0])
    ntiles = nc.createDimension('ntiles', 1)
    z = nc.createVariable('depth', float, ('ny','nx') )
    z.units='meters'
    z2 = nc.createVariable('h2', float, ('ny','nx'))
    z2.units='meters^2'
    z[:,:] = -Htarg[:,:]
    z2[:,:] = H2targ[:,:]

In [15]:
!ls new_topo*

new_topo_OM5_grid_r8_12x43.nc


In [16]:
with netCDF4.Dataset('new_topo_OM5_grid_r{}_{}x{}.nc'.format(max_refinement, NtileI, NtileJ)) as nc:
    depth = nc.variables['depth'][:,:]
    roughness = nc.variables['h2'][:,:]

In [17]:
#Compare with the raw results of OM5p25v20240311
with netCDF4.Dataset('.datasets/OM5p25v20240311_jpk/new_topo_OM5_grid_r8_24x387.nc') as nc:
    depth_OM5 = nc.variables['depth'][:,:]
    rough_OM5 = nc.variables['h2'][:,:]


In [ ]:
depth_diff=depth-depth_OM5
plt.figure(figsize=(14,8)); 
plt.subplot(121); plt.pcolormesh( depth); plt.colorbar();
plt.subplot(122); plt.pcolormesh( depth_diff); plt.colorbar();


In [23]:
depth_diff.max(),depth_diff.min()

(60.25144958496094, -70.78657531738281)

In [ ]:
rough_diff=roughness-rough_OM5
plt.figure(figsize=(14,8)); 
plt.subplot(121); plt.pcolormesh(roughness); plt.colorbar();
plt.subplot(122); plt.pcolormesh(rough_diff); plt.colorbar();

In [28]:
roughness.max(),rough_diff.min(),rough_diff.max()

(1839942.65612669, -63947.503729308635, 74260.39504513051)

In [ ]:
plt.figure(figsize=(14,8)); 
plt.pcolormesh( depth_diff,vmin=-1,vmax=1); plt.colorbar();

In [22]:
depth.max()

9671.759765625